In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
from dask.distributed import Client, progress

client = Client()
# client = Client('127.0.0.1:8786')

client

Client Scheduler: tcp://127.0.0.1:56567 Dashboard: http://127.0.0.1:56568/status,Cluster Workers: 8 Cores: 8 Memory: 17.18 GB


In [3]:
import pandas as pd

import dask_ml.joblib

from sklearn.svm import SVC
from sklearn.datasets import make_classification
from sklearn.model_selection import GridSearchCV

In [4]:
X, y = make_classification(n_samples=1000, random_state=0)
X[:5]

array([[-1.06377997,  0.67640868,  1.06935647, -0.21758002,  0.46021477,
        -0.39916689, -0.07918751,  1.20938491, -0.78531472, -0.17218611,
        -1.08535744, -0.99311895,  0.30693511,  0.06405769, -1.0542328 ,
        -0.52749607, -0.0741832 , -0.35562842,  1.05721416, -0.90259159],
       [ 0.0708476 , -1.69528125,  2.44944917, -0.5304942 , -0.93296221,
         2.86520354,  2.43572851, -1.61850016,  1.30071691,  0.34840246,
         0.54493439,  0.22532411,  0.60556322, -0.19210097, -0.06802699,
         0.9716812 , -1.79204799,  0.01708348, -0.37566904, -0.62323644],
       [ 0.94028404, -0.49214582,  0.67795602, -0.22775445,  1.40175261,
         1.23165333, -0.77746425,  0.01561602,  1.33171299,  1.08477266,
        -0.97805157, -0.05012039,  0.94838552, -0.17342825, -0.47767184,
         0.76089649,  1.00115812, -0.06946407,  1.35904607, -1.18958963],
       [-0.29951677,  0.75988955,  0.18280267, -1.55023271,  0.33821802,
         0.36324148, -2.10052547, -0.4380675 , -

In [5]:
param_grid = {
    'C': [0.001, 0.01, 0.1, 0.5, 1.0, 2.0, 5.0, 10.0],
    'kernel': ['rbf', 'poly', 'sigmoid'],
    'shrinking': [True, False]
}

grid_search = GridSearchCV(SVC(gamma='auto', random_state=0, probability=True), 
                           param_grid=param_grid, return_train_score=False, iid=True, n_jobs=-1)

In [6]:
from sklearn.externals import joblib

with joblib.parallel_backend('dask', scatter=[X, y]):
    grid_search.fit(X, y)

In [7]:
pd.DataFrame(grid_search.cv_results_).head()

,mean_fit_time,mean_score_time,mean_test_score,param_C,param_kernel,param_shrinking,params,rank_test_score,split0_test_score,split1_test_score,split2_test_score,std_fit_time,std_score_time,std_test_score
0,0.359381,0.028917,0.502,0.001,rbf,True,"{'C': 0.001, 'kernel': 'rbf', 'shrinking': True}",41,0.502994,0.501502,0.501502,0.070795,0.016370,0.000704
1,0.326675,0.025953,0.502,0.001,rbf,False,"{'C': 0.001, 'kernel': 'rbf', 'shrinking': False}",41,0.502994,0.501502,0.501502,0.023104,0.012870,0.000704
2,0.254754,0.011779,0.502,0.001,poly,True,"{'C': 0.001, 'kernel': 'poly', 'shrinking': True}",41,0.502994,0.501502,0.501502,0.040616,0.006726,0.000704
3,0.133559,0.024178,0.502,0.001,poly,False,"{'C': 0.001, 'kernel': 'poly', 'shrinking': Fa...",41,0.502994,0.501502,0.501502,0.011694,0.017335,0.000704
4,0.306559,0.020773,0.502,0.001,sigmoid,True,"{'C': 0.001, 'kernel': 'sigmoid', 'shrinking':...",41,0.502994,0.501502,0.501502,0.016354,0.011464,0.000704
